In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from ConvNeXt import Block, ConvNeXt, LayerNorm

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/asavin/.local/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
data = torch.rand([32, 3, 40, 120])

In [4]:
network = ConvNeXt(num_classes=32)

In [5]:
print(network)

ConvNeXt(
  (downsample_layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm()
    )
    (1): Sequential(
      (0): LayerNorm()
      (1): Conv2d(96, 192, kernel_size=(2, 2), stride=(2, 2))
    )
    (2): Sequential(
      (0): LayerNorm()
      (1): Conv2d(192, 384, kernel_size=(2, 2), stride=(2, 2))
    )
    (3): Sequential(
      (0): LayerNorm()
      (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
    )
  )
  (stages): ModuleList(
    (0): Sequential(
      (0): Block(
        (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
        (norm): LayerNorm()
        (pwconv1): Linear(in_features=96, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (pwconv2): Linear(in_features=384, out_features=96, bias=True)
        (drop_path): Identity()
      )
      (1): Block(
        (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(

In [6]:
out = network.forward(data)

In [7]:
out.shape

torch.Size([32, 32])

In [8]:
network2 = ConvNeXt(num_classes=32)

In [9]:
network2.norm = nn.Identity()
network2.head = nn.Identity()

In [10]:
out2 = network2.forward(data)

In [11]:
out2.shape

torch.Size([32, 768])

In [12]:
print(network.head)

Linear(in_features=768, out_features=32, bias=True)


In [13]:
print(network.stages[-1][2])

Block(
  (dwconv): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
  (norm): LayerNorm()
  (pwconv1): Linear(in_features=768, out_features=3072, bias=True)
  (act): GELU(approximate='none')
  (pwconv2): Linear(in_features=3072, out_features=768, bias=True)
  (drop_path): Identity()
)


In [14]:
norm1 = LayerNorm(3, eps=1e-6)

In [15]:
norm2 = LayerNorm(3, eps=1e-6, data_format="channels_first")

In [16]:
norm2(data).shape

torch.Size([32, 3, 40, 120])

In [17]:
norm1(data.permute(0, 2, 3, 1)).shape

torch.Size([32, 40, 120, 3])

In [18]:
lin = nn.Linear(3, 4 * 3)

In [19]:
lin(norm1(data.permute(0, 2, 3, 1))).shape

torch.Size([32, 40, 120, 12])